#### Build a Seq2Seq Language Translation model for any language pair of your choice. (See Datasets: http://www.manythings.org/anki/ )

(1) Build a character-level model 

(2) Build a word level model

(3) Build a word model with attention

Submit code and pdf. Include sample inferences from best performing model 

### 2. Word level model
#### Dataset used : French Language (fra.text)

#### Preprocessing the text

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.python.keras.models import Model

In [3]:
line= pd.read_table('fra.txt', nrows=10000, usecols=[0,1], names=['eng', 'fre'])

line.eng=line.eng.apply(lambda x: x.lower())
line.fre=line.fre.apply(lambda x: x.lower())

line.eng=line.eng.apply(lambda x: re.sub("'", '', str(x)))
line.fre=line.fre.apply(lambda x: re.sub("'", '', str(x)))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
line.eng=line.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
line.fre=line.fre.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
line.fre = line.fre.apply(lambda x : 'START_ '+ x + ' _END')

remove_digits = str.maketrans('', '', string.digits)
line.eng=line.eng.apply(lambda x: x.translate(remove_digits))


#### Create the Word List

In [4]:
# Vocabulary of English
all_eng_words=set()
for eng in line.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

            
all_french_words=set() # will have only unique words of French
for fre in line.fre:
    for word in fre.split():
        if word not in all_french_words:
            all_french_words.add(word)

# Max Length of source sequence
lenght_list=[]
for l in line.eng:
    lenght_list.append(len(l.split(' ')))
    
max_length_src = np.max(lenght_list)
max_length_src

#Max Length of target sequence
lenght_list=[]
for l in line.fre:
    lenght_list.append(len(l.split(' ')))
    
max_length_tar = np.max(lenght_list)
max_length_tar

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
num_encoder_tokens, num_decoder_tokens

num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

4568

In [5]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
line.head()

,eng,fre
0,go,START_ va _END
1,hi,START_ salut _END
2,hi,START_ salut _END
3,run,START_ cours _END
4,run,START_ courez _END


In [6]:
# split train and test data
X, Y = line.eng, line.fre
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
X_train.shape, X_test.shape

((8000,), (2000,))

In [7]:
#Generating Zero arrays
def generate_batch(X = X_train, Y = Y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], Y[j:j+batch_size])):  
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

latent_dim = 50

### Define the word level model

In [8]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [9]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [10]:
#Compile the Model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     107900      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     228450      input_2[0][0]                    
_______________________________________________________________________________________

In [12]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [13]:
# Fit the Model
model.fit_generator(generator = generate_batch(X_train, Y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, Y_test, batch_size =batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
62/62 [==============================] - 26s 416ms/step - loss: 2.3572 - acc: 0.2331 - val_loss: 1.9308 - val_acc: 0.2362
Epoch 2/100
62/62 [==============================] - 27s 433ms/step - loss: 1.8557 - acc: 0.2378 - val_loss: 1.9105 - val_acc: 0.2362
Epoch 3/100
62/62 [==============================] - 33s 540ms/step - loss: 1.8238 - acc: 0.2377 - val_loss: 1.9135 - val_acc: 0.2362
Epoch 4/100
62/62 [==============================] - 29s 463ms/step - loss: 1.7928 - acc: 0.2451 - val_loss: 1.8899 - val_acc: 0.2703
Epoch 5/100
62/62 [==============================] - 26s 417ms/step - loss: 1.7249 - acc: 0.2800 - val_loss: 1.7968 - val_acc: 0.2832
Epoch 6/100
62/62 [==============================] - 25s 408ms/step - loss: 1.6347 - acc: 0.2946 - val_loss: 1.7235 - val_acc: 0.3025
Epoch 7/100
62/62 [==============================] - 28s 451ms/step - loss: 1.5701 - acc: 0.3081 - val_loss: 1.6782 - va

### Inference set up

In [14]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [15]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Sample Predictions :

In [16]:
train_gen = generate_batch(X_train, Y_train, batch_size = 1)
k=-1

In [17]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: youre in luck
Actual Target Translation:  vous êtes veinards 
Predicted Target Translation:  vous êtes veinard 


In [19]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: use them all
Actual French Translation:  utilisezles toutes  
Predicted French Translation:  utilisezles toutes 


In [20]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual French Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted French Translation:', decoded_sentence[:-4])

Input English sentence: once again
Actual French Translation:  encore une fois 
Predicted French Translation:  encore une fois 


### Observations :
* The Seq2seq Model built using the Word-based model gives a Validation Accuracy of 73%.
* The Accuracy of this Model is much higher in comparison to the character-based seq2seq Translation Model, which only gave an accuracy of 22%.
* The word-based model is accurately predicting single words. 
* It also seems to be performing quite well even on successive words in small sentences, as can be seen in the above sample predictions.

=> Thus, based on these observations we can infer that the word-based model is more reliable and efficient for Language Translation features.
